In [1]:
import pandas as pd

In [2]:
csv_data = pd.read_csv(
    './assets/IHME-GBD_2017_DATA-37d305ef-1.csv',
    error_bad_lines=False
)
df = pd.DataFrame(csv_data)

In [3]:
del df['measure_id']
del df['measure_name']
df['location_id'] = df['location_id'].fillna(0)
df['location_name'] = df['location_name'].fillna('unknown')
df['sex_id'] = df['sex_id'].fillna(0)
df['sex_name'] = df['sex_name'].fillna('unknown')
del df['age_id']
del df['age_name']
del df['cause_id']
del df['cause_name']
del df['metric_id']
del df['metric_name']
df['year'] = df['year'].fillna(0)
df['val'] = df['val'].fillna(0)
df['upper'] = df['upper'].fillna(0)
df['lower'] = df['lower'].fillna(0)
df['rank'] = 0
df['percent_change'] = 0.0
df['average_percent_change'] = 0.0
# df['current_to_peak'] = 0.0
df['raw_decrease_from_peak'] = 0.0
df['raw_increase_from_min'] = 0.0
# df['current_to_min'] = 0.0
# df['raw_increase'] = 0.0
df['avg_percent_change_since_peak'] = 0.0
df['avg_percent_change_since_min'] = 0.0
df['peak'] = 0.0
df['min'] = 0.0

In [4]:
df[(df['sex_id']==3) & (df['year']==1990)].sort_values(by=['val'], ascending=False).reset_index().head()

,index,location_id,location_name,sex_id,sex_name,year,val,upper,lower,rank,percent_change,average_percent_change,raw_decrease_from_peak,raw_increase_from_min,avg_percent_change_since_peak,avg_percent_change_since_min,peak,min
0,443,196,South Africa,3,Both,1990,4.929720,5.472670,4.487900,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,431,94,Switzerland,3,Both,1990,3.566200,4.055893,3.002744,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,362,62,Russian Federation,3,Both,1990,2.371923,2.504497,2.214237,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1283,71,Australia,3,Both,1990,1.892743,2.028607,1.758087,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,215,90,Norway,3,Both,1990,1.648328,1.725415,1.548719,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df2 = df[(df['sex_id']==3) & (df['year']==1990)].sort_values(by=['val'], ascending=False).reset_index().head()

In [6]:
for year in df['year'].unique(): 
    for sex in df['sex_id'].unique():
        df_current = df[(df['sex_id']==sex) & (df['year']==year)].sort_values(by=['val'], ascending=False).reset_index()
        for index, row in df_current.iterrows():
              df.at[row['index'], 'rank'] = index + 1

In [7]:
for country in df['location_name'].unique():
    for sex in df['sex_id'].unique():
        df_country = df[(df['sex_id']==sex) & (df['location_name']==country)].sort_values(by=['year'])
        previous_val = df_country[(df_country['year']==1990)]['val']
        year_of_peak =  df_country.loc[df_country['val'].idxmax()]['year']
        year_of_min =  df_country.loc[df_country['val'].idxmin()]['year']
        total_change = 0
        total_change_since_peak = 0
        total_change_since_min = 0
        years_since_peak = 0
        years_since_min = 0
        for index, row in df_country[1:].iterrows():
            percent_change = (row['val'] - previous_val)/previous_val
            total_change += percent_change
            if(row['year'] > year_of_peak):
                total_change_since_peak += percent_change
                years_since_peak+=1
            if(row['year'] > year_of_min):
                total_change_since_min += percent_change
                years_since_min+=1
            previous_val = row['val']
            df.at[index, 'percent_change'] = percent_change
        df.at[index, 'average_percent_change'] = total_change/26
        try:
            df.at[index, 'avg_percent_change_since_peak'] = total_change_since_peak/(years_since_peak)
        except ZeroDivisionError:
            pass
        try:
            df.at[index, 'avg_percent_change_since_min'] = total_change_since_min/(years_since_min)
        except ZeroDivisionError:
            pass
        
        df.at[index, 'peak'] = df_country['val'].max()
        df.at[index, 'min'] = df_country['val'].min()
        df.at[index, 'current_to_peak'] = (float(df_country[(df['year']==2017)]['val']) - df_country['val'].max())/df_country['val'].max()
        df.at[index, 'raw_decrease_from_peak'] = float(df_country[(df['year']==2017)]['val']) - df_country['val'].max()
        df.at[index, 'raw_increase_from_min'] = float(df_country[(df['year']==2017)]['val']) - df_country['val'].min()
#         df.at[index, 'current_to_min'] = (float(df_country[(df['year']==2017)]['val']) - df_country['val'].min())/df_country['val'].min()
#         df.at[index, 'raw_increase'] = float(df_country[(df['year']==2017)]['val']) - float(df_country[(df['year']==1990)]['val'])
# 
        

/Users/benjaminhurst/.local/share/virtualenvs/opioid_study-oZUPmX6j/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/benjaminhurst/.local/share/virtualenvs/opioid_study-oZUPmX6j/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/benjaminhurst/.local/share/virtualenvs/opioid_study-oZUPmX6j/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [152]:
df_country.loc[df_country['val'].idxmin()]['year']

1990

In [8]:
 df_current = df[(df['sex_id']==1) & (df['location_id']==102)].sort_values(by=['year'])

In [167]:
df[(df['year']==2017) & (df['sex_id']==1) & (df['peak'] > 4) & (df['raw_decrease_from_peak'] < -2)].sort_values(by=['avg_percent_change_since_peak'])


,location_id,location_name,sex_id,sex_name,year,val,upper,lower,rank,percent_change,average_percent_change,current_to_peak,raw_decrease_from_peak,raw_increase_from_min,current_to_min,raw_increase,avg_percent_change_since_peak,avg_percent_change_since_min,peak,min
16215,196,South Africa,1,Male,2017,0.908415,1.058998,0.790352,109,0.224945,-0.065616,-0.919730,-10.408528,0.318043,0.538716,-6.528748,-0.107567,0.156841,11.316943,0.590372
15510,151,Qatar,1,Male,2017,0.284638,0.370529,0.211766,176,0.025445,-0.054061,-0.932821,-3.952372,0.065725,0.300232,-1.703746,-0.102154,0.036147,4.237010,0.218913
16002,385,Puerto Rico,1,Male,2017,2.575941,3.074977,2.147108,27,0.092242,0.059002,-0.726595,-6.845754,1.598081,1.634263,1.598081,-0.061643,0.056817,9.421695,0.977860
15969,82,Greece,1,Male,2017,2.534104,2.913368,2.197369,28,0.003343,0.063892,-0.543935,-3.022345,1.875149,2.845639,1.875149,-0.061150,0.061526,5.556449,0.658955
16323,94,Switzerland,1,Male,2017,1.495854,1.855795,1.192823,56,0.031203,-0.047833,-0.775491,-5.166936,0.085692,0.060768,-4.364162,-0.059842,0.011983,6.662790,1.410161
16374,90,Norway,1,Male,2017,3.878455,4.333086,3.520894,15,0.049647,0.019678,-0.548960,-4.720459,1.218489,0.458084,1.218489,-0.043934,0.018949,8.598915,2.659966
15804,58,Estonia,1,Male,2017,9.241272,11.826652,7.275460,3,-0.021291,0.069104,-0.196086,-2.254070,7.415962,4.062851,7.415962,-0.040970,0.066544,11.495342,1.825310
16116,62,Russian Federation,1,Male,2017,7.477044,8.023515,7.012549,4,0.025136,0.028326,-0.389801,-4.776400,3.371633,0.821266,3.371633,-0.039435,0.027277,12.253444,4.105411


In [168]:
df[(df['year']==2017) & (df['sex_id']==2) & (df['peak'] > 1) & (df['raw_decrease_from_peak'] < -.3)].sort_values(by=['avg_percent_change_since_peak'])



,location_id,location_name,sex_id,sex_name,year,val,upper,lower,rank,percent_change,average_percent_change,current_to_peak,raw_decrease_from_peak,raw_increase_from_min,current_to_min,raw_increase,avg_percent_change_since_peak,avg_percent_change_since_min,peak,min
16216,196,South Africa,2,Female,2017,0.652536,0.771763,0.570685,37,0.098261,-0.043149,-0.840946,-3.450076,0.126507,0.240495,-1.936238,-0.096063,0.075048,4.102612,0.526029
16324,94,Switzerland,2,Female,2017,0.494604,0.580558,0.423454,55,0.051927,-0.033498,-0.665605,-0.984498,0.058685,0.134624,-0.836057,-0.042539,0.043043,1.479102,0.435919
16312,37,Kyrgyzstan,2,Female,2017,0.707965,0.796314,0.638144,32,0.041201,0.026930,-0.340353,-0.365283,0.329909,0.872648,0.329909,-0.036087,0.025933,1.073248,0.378056
16375,90,Norway,2,Female,2017,1.145596,1.295023,1.052265,12,0.047857,0.025787,-0.410017,-0.796146,0.486912,0.739219,0.486912,-0.030107,0.024832,1.941742,0.658684
16366,142,Iran,2,Female,2017,1.167278,1.265283,1.085196,10,0.028670,0.031636,-0.206138,-0.303101,0.620564,1.135080,0.620564,-0.029657,0.030464,1.470379,0.546714
16117,62,Russian Federation,2,Female,2017,1.755357,1.875963,1.666198,6,0.036396,0.032842,-0.234590,-0.537999,0.911943,1.081252,0.911943,-0.021333,0.031625,2.293356,0.843414


In [171]:
df[(df['year']==2017) & (df['sex_id']==3) & (df['peak'] > 2) & (df['raw_decrease_from_peak'] < -1.5)].sort_values(by=['avg_percent_change_since_peak'])



,location_id,location_name,sex_id,sex_name,year,val,upper,lower,rank,percent_change,average_percent_change,current_to_peak,raw_decrease_from_peak,raw_increase_from_min,current_to_min,raw_increase,avg_percent_change_since_peak,avg_percent_change_since_min,peak,min
16217,196,South Africa,3,Both,2017,0.777677,0.896876,0.693372,84,0.167277,-0.058778,-0.892802,-6.476925,0.220244,0.395104,-4.152043,-0.097118,0.118780,7.254602,0.557433
15512,151,Qatar,3,Both,2017,0.238284,0.304725,0.181966,169,0.026992,-0.048419,-0.916734,-2.623433,0.060589,0.340968,-1.139563,-0.094071,0.039738,2.861717,0.177696
15971,82,Greece,3,Both,2017,1.474261,1.666791,1.304894,31,0.007922,0.061266,-0.523968,-1.622720,1.072560,2.670044,1.072560,-0.057872,0.058997,3.096981,0.401701
16325,94,Switzerland,3,Both,2017,0.992708,1.182914,0.841801,53,0.036863,-0.044821,-0.753895,-3.040966,0.063643,0.068503,-2.573491,-0.056177,0.013513,4.033674,0.929065
16004,385,Puerto Rico,3,Both,2017,1.627778,1.874211,1.415452,25,0.099540,0.059493,-0.670257,-3.308727,1.052813,1.831092,1.052813,-0.052468,0.057290,4.936505,0.574965
16376,90,Norway,3,Both,2017,2.522687,2.805491,2.340844,15,0.049710,0.021195,-0.515433,-2.683379,0.874359,0.530452,0.874359,-0.040028,0.020410,5.206066,1.648328
16118,62,Russian Federation,3,Both,2017,4.415154,4.710736,4.193987,5,0.027842,0.028978,-0.362494,-2.510508,2.043231,0.861424,2.043231,-0.035983,0.027905,6.925662,2.371923


In [157]:
df[(df['year']==2017) & (df['sex_id']==1) & (df['val'] > 4)].sort_values(by=['avg_percent_change_since_min'], ascending=False)



,location_id,location_name,sex_id,sex_name,year,val,upper,lower,rank,percent_change,average_percent_change,current_to_peak,raw_decrease_from_peak,raw_increase_from_min,current_to_min,raw_increase,avg_percent_change_since_peak,avg_percent_change_since_min,peak,min
15471,102,United States,1,Male,2017,19.550164,20.962672,18.438986,1,0.053881,0.093421,0.000000,0.000000,17.609666,9.074819,17.609666,0.000000,0.089961,19.550164,1.940498
15636,36,Kazakhstan,1,Male,2017,5.232632,6.118458,4.463960,9,0.067408,0.070455,-0.269917,-1.934538,4.235426,4.247290,4.235426,-0.027469,0.067845,7.167170,0.997207
15804,58,Estonia,1,Male,2017,9.241272,11.826652,7.275460,3,-0.021291,0.069104,-0.196086,-2.254070,7.415962,4.062851,7.415962,-0.040970,0.066544,11.495342,1.825310
16314,147,Libya,1,Male,2017,11.047638,15.174900,7.619245,2,0.023466,0.061453,-0.000335,-0.003697,8.658221,3.623570,8.658221,0.000106,0.059177,11.051336,2.389417
16086,60,Lithuania,1,Male,2017,5.620137,6.479079,4.823678,7,-0.014109,0.053229,-0.014109,-0.080429,4.075232,2.637853,4.075232,-0.014109,0.051257,5.700565,1.544905
15900,101,Canada,1,Male,2017,5.008996,5.827296,4.250928,11,0.011486,0.050611,0.000000,0.000000,3.587974,2.524926,3.587974,0.000000,0.048737,5.008996,1.421022
15612,156,United Arab Emirates,1,Male,2017,7.173153,9.714108,5.031084,5,0.025612,0.048133,0.000000,0.000000,5.040189,2.362998,5.040189,0.000000,0.046350,7.173153,2.132964
16341,146,Lebanon,1,Male,2017,5.195962,6.575632,4.065280,10,0.028697,0.046595,0.000000,0.000000,3.605752,2.267470,3.605752,0.000000,0.044870,5.195962,1.590210
16365,142,Iran,1,Male,2017,5.915981,6.518336,5.582490,6,0.014398,0.043043,-0.158381,-1.113307,3.814823,1.815582,3.814823,-0.025743,0.041448,7.029289,2.101158
16209,71,Australia,1,Male,2017,4.100338,4.947907,3.378137,14,0.029140,0.015895,-0.253218,-1.390335,1.308936,0.468917,1.147183,-0.012474,0.035964,5.490674,2.791403


In [169]:
df[(df['year']==2017) & (df['sex_id']==2) & (df['val'] > 1)].sort_values(by=['avg_percent_change_since_min'], ascending=False)



,location_id,location_name,sex_id,sex_name,year,val,upper,lower,rank,percent_change,average_percent_change,current_to_peak,raw_decrease_from_peak,raw_increase_from_min,current_to_min,raw_increase,avg_percent_change_since_peak,avg_percent_change_since_min,peak,min
15472,102,United States,2,Female,2017,9.749631,11.347451,7.986645,1,0.050149,0.117621,0.000000,0.000000,9.204797,16.894693,9.204797,0.000000,0.113265,9.749631,0.544834
16339,95,United Kingdom,2,Female,2017,1.091798,1.224378,1.032514,15,-0.003663,0.069548,-0.003663,-0.004014,0.881933,4.202388,0.881933,-0.003663,0.066972,1.095811,0.209865
15613,156,United Arab Emirates,2,Female,2017,1.761802,2.466719,1.203638,5,0.049080,0.062537,0.000000,0.000000,1.396214,3.819095,1.396214,0.000000,0.060221,1.761802,0.365588
16315,147,Libya,2,Female,2017,3.304300,4.377883,2.292456,2,0.048871,0.060944,0.000000,0.000000,2.581423,3.571039,2.581423,0.000000,0.058687,3.304300,0.722877
15901,101,Canada,2,Female,2017,2.180181,2.475569,1.905558,3,0.014913,0.057171,0.000000,0.000000,1.654731,3.149169,1.654731,0.000000,0.055053,2.180181,0.525450
15805,58,Estonia,2,Female,2017,1.103492,1.393087,0.866087,13,-0.038773,0.055266,-0.154572,-0.201754,0.774046,2.349539,0.774046,-0.041052,0.053219,1.305246,0.329446
15637,36,Kazakhstan,2,Female,2017,1.097559,1.247706,0.966640,14,0.030159,0.054202,-0.195534,-0.266774,0.799719,2.685060,0.799719,-0.019492,0.052195,1.364333,0.297840
16156,139,Algeria,2,Female,2017,1.197644,1.429624,0.972024,9,0.024450,0.052053,0.000000,0.000000,0.876437,2.728572,0.876437,0.000000,0.050125,1.197644,0.321207
15526,57,Belarus,2,Female,2017,1.362100,1.568958,1.197107,7,0.051049,0.047299,0.000000,0.000000,0.934217,2.183344,0.934217,0.000000,0.045547,1.362100,0.427883
16117,62,Russian Federation,2,Female,2017,1.755357,1.875963,1.666198,6,0.036396,0.032842,-0.234590,-0.537999,0.911943,1.081252,0.911943,-0.021333,0.031625,2.293356,0.843414


In [174]:
df[(df['year']==2017) & (df['sex_id']==3) & (df['val'] > 2.5)].sort_values(by=['avg_percent_change_since_min'], ascending=False)



,location_id,location_name,sex_id,sex_name,year,val,upper,lower,rank,percent_change,average_percent_change,current_to_peak,raw_decrease_from_peak,raw_increase_from_min,current_to_min,raw_increase,avg_percent_change_since_peak,avg_percent_change_since_min,peak,min
15473,102,United States,3,Both,2017,14.574242,15.635498,13.509508,1,0.052565,0.100208,0.000000,0.000000,13.347946,10.884766,13.347946,0.000000,0.096497,14.574242,1.226296
15638,36,Kazakhstan,3,Both,2017,3.097606,3.551099,2.706371,9,0.060771,0.066797,-0.254921,-1.059811,2.460360,3.860922,2.460360,-0.025846,0.064323,4.157417,0.637247
15806,58,Estonia,3,Both,2017,4.923354,6.202291,3.955850,4,-0.022136,0.066336,-0.185868,-1.124012,3.894427,3.784941,3.894427,-0.039043,0.063879,6.047365,1.028927
16316,147,Libya,3,Both,2017,7.293058,9.564078,5.409147,2,0.028508,0.060943,0.000000,0.000000,5.700854,3.580480,5.700854,0.000000,0.058686,7.293058,1.592204
15614,156,United Arab Emirates,3,Both,2017,5.726530,7.576543,4.133446,3,0.025668,0.052976,0.000000,0.000000,4.218992,2.798599,4.218992,0.000000,0.051014,5.726530,1.507537
15902,101,Canada,3,Both,2017,3.571801,3.998376,3.172836,7,0.012653,0.052277,0.000000,0.000000,2.604534,2.692675,2.604534,0.000000,0.050341,3.571801,0.967266
16088,60,Lithuania,3,Both,2017,2.985168,3.377117,2.609929,10,-0.010996,0.050039,-0.010996,-0.033191,2.089317,2.332217,2.089317,-0.010996,0.048185,3.018359,0.895850
16343,146,Lebanon,3,Both,2017,3.524889,4.238569,2.872015,8,0.032333,0.042582,0.000000,0.000000,2.332862,1.957053,2.332862,0.000000,0.041005,3.524889,1.192028
16367,142,Iran,3,Both,2017,3.575545,3.910082,3.399822,6,0.017170,0.040697,-0.156394,-0.662860,2.236837,1.670892,2.236837,-0.021687,0.039190,4.238406,1.338708
16211,71,Australia,3,Both,2017,2.712225,3.127491,2.341691,13,0.030894,0.016876,-0.210471,-0.723020,0.879427,0.479828,0.819482,-0.009758,0.036555,3.435244,1.832797
